<a href="https://colab.research.google.com/github/nagasaichukkapalli/Airbnb-Booking-analysis-EDA/blob/main/nagasai_Airbnb_booking_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Airbnb EDA Project by NAGA SAI KIRAN CHUKKAPALLI**

## Okay, before going to start. Let's understand what is Airbnb?

Looks Airbnb has interesting breakdown with names likely: Air, Bed and Breakfast to become Airbnb. Wow! This San-Francisco based startup offers you someone’s home as a place to stay instead of a hotel. Looks, somewhat on a same business as OYO but the former doesn't owns any property instead acts as an intermediary between those who want to rent out space and those who are looking for space to rent.

Well, enough of it we understood what is the data all about and where it came from.


> **Breakdown of this Notebook :**

  1.  **Importing Libraries**

  2.  **Loading the dataset**

  1.  **Dataset Information**

  2.  **Data Cleaning:** 

      *   Unique values For Variable Identification.

      *   Checking For duplicates.
      *   Summary of Statistics.
      *   Missing Values Treatment
      *   Skewness and kurtosis.
      *   Outliner Detection & Treatment

  1.   **Data Analysis and Visualization:** Using plots to find relations between the features
      * Univariate Analysis
        *   Numerical Analysis
      *   Bivariate Analysis

      *   Multivariate Analysis
    







6.   **Conclusion**















In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/cap stone-1/Copy of Airbnb NYC 2019.csv")

In [ ]:
df.head()

In [ ]:
#check the volume of data
df.shape


In [ ]:
#checking the information about column wise number of non null records & type of columns in the given data
df.info()

In [ ]:
#checking unique values for all columns( it is imp for knowing the categorical and numerical variables for analysing the data)
df.describe(include='all').loc['unique', :]

In [ ]:
duplicate_data = df.duplicated()
print(duplicate_data.sum())
df[duplicate_data]
#looks like there's no duplicate data present!

#  **creating a Subset of Duplicated Data**

In [ ]:
print(df.duplicated(subset=['name']).sum())

In [ ]:
#finding duplicate values of name variable in the given data
df.duplicated(subset=['name']).sum()

In [ ]:
#duplicate records containing same name
df.loc[df.duplicated(subset=['name']), :].head()

In [ ]:
#duplicate records containing same name of Loft w/ Terrace @ Box House Hotel
df.loc[df['name']=='Loft w/ Terrace @ Box House Hotel']

In [ ]:
df.loc[df['host_name']=='Natasha']

From the above data Host id and host names are linked and as from the above data one host has many host ids with respect to change in neighbourhood groups and neighbourhood and listing names

#  Statistics Summary

In [ ]:
#getting to know the description of data.( most of time we can get to know about ouliners here by min,max and quartiles). price seems to have outlines as min 0 means it is free and max 10,000 but median is 106
df.describe().round(2)

From the above information we can observe that the host id is unique id and we can skip that on this features like min,max and main important point is price is having outliners by measuring range and observing inter quartile ranges

In [ ]:
len(df[df['price']==0])

In [ ]:
df[df['price']==0]

# Checking Null Values

In [ ]:
df.isnull().sum()

# Missing Values Treatment

In [ ]:
df['name'].fillna('dontknown',inplace=True) #filling nan with dontknown as we dont know the name
df['host_name'].fillna('no_name',inplace=True) #filling nan with no_name as we dont know the hostname
df['reviews_per_month']=df['reviews_per_month'].replace(to_replace=np.nan,value=0).astype('int64')#filling nan with 0 and as type integer beacause it is quantitative
df.drop(['id','last_review'],axis=1,inplace=True) #last review it is unnecessary we can drop the column

##Checking For any null values remaining

In [ ]:
df.info()

# Skewness and kurtosis

In [ ]:
df.skew()

In [ ]:
df.kurt()

# **Outliner Detection & Treatment**
##Univariate outliers:

In [ ]:
sns.set(rc={"figure.figsize": (20, 6)})
sns.distplot(df['price'],hist=True)

The distribution tells it has positive skew also the distribution doesn't deviate much from normal distribution.

Since the skewness has value more than 1 it is highly skewed.

Also kurtosis looks high as well which indicates presence of more amount of outliers,we will look after into that when we handle outliers.

In [ ]:
df.boxplot(column=['price'])
plt.show()
#we can see the presence of many outliers in price.

# **Multivariate outliers:**

In [ ]:
# price vs minimum_nights

mn='minimum_nights'

data=pd.concat([df['price'],df[mn]],axis=1)
data.plot.scatter(x=mn,y='price',ylim=(0,12000))

looks many data points are clustured on 0 price range, few have min nights for stay but price is 0. looks like anomaly in price.

there are many outliers as well.

#Outliners Treatment
When the data is skewed, it is good to consider using the median value for replacing the missing values.

In [ ]:
len(df[df['price']==0])

In [ ]:
df['price'].median()

In [ ]:
newdf = df.copy()
print(newdf)

In [ ]:
newdf['price'] = np.where(
   (newdf['price'] == 0) & (newdf['minimum_nights'] == 1) ,round(df['price'].median()) , newdf['price'])

In [ ]:
newdf.describe()

In [ ]:
len(newdf[newdf['price']==0])

In [ ]:
#Function that gives a new price according to minimum night
def price_calculator(min_night):
  median_price = round(df['price'].median())
  new_price = median_price * min_night

  return new_price

In [ ]:
#Replace the value of price according to minimum night where price = 0$ 
newdf['price'] = np.where(
   (newdf['price'] == 0) ,price_calculator(newdf['minimum_nights']) , newdf['price'])

In [ ]:
newdf.describe()

In [ ]:
len(newdf[newdf['price']==0])

In [ ]:
df['price'].mean()

In [ ]:
newdf['price'].mean()

# Outliners Removal by Quantile method

In [ ]:
min_threshold,max_threshold= newdf.price.quantile([0.01,0.999])
min_threshold,max_threshold

In [ ]:
#prices less than min threshold!
newdf[newdf.price<min_threshold]

In [ ]:
newdf[newdf.price>max_threshold]
#similary for getting extreme price value above max_threshold

In [ ]:
quantdf= newdf[(newdf.price>min_threshold)&(newdf.price<max_threshold)]

In [ ]:
sns.distplot(quantdf['price'])

In [ ]:
sns.boxplot(quantdf['price']) #looks there are still many outliers!

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % quantdf['price'].skew())
print("Kurtosis: %f" % quantdf['price'].kurt())

# Outliners Removal by IQR method

In [ ]:
Q1 = newdf.quantile(0.25)
Q3 = newdf.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
#writing a outlier function for removing outliers!
def outlier_treatment(datacolumn):
 sorted(datacolumn)
 Q1,Q3 = np.percentile(datacolumn , [25,75])
 IQR = Q3 - Q1
 lower_range = Q1 - (1.5 * IQR)
 upper_range = Q3 + (1.5 * IQR)
 
 return lower_range,upper_range

In [ ]:
lower_bound,upper_bound = outlier_treatment(newdf['price'])
lower_bound,upper_bound

In [ ]:
newdf[(newdf.price < lower_bound) | (newdf.price > upper_bound)] #looking at the low and upper range outliers

In [ ]:
#subsetting only those rows having datapoints greater than lower range and lesser than upper range respectively
iqrdf = newdf[(newdf.price>lower_bound) & (newdf.price<upper_bound)]

In [ ]:
iqrdf

In [ ]:
iqrdf.boxplot(column='price') #the boxplot for price now looks good without outliers!

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % iqrdf['price'].skew())
print("Kurtosis: %f" % iqrdf['price'].kurt())

In [ ]:
sns.distplot(iqrdf['price'])

Iqr method did the best and as we can see the skewness is less than 1 and this df will be used to find the price distribution on remaining features as this df will not have any impact from outliners

#Data Analysis
Let's check the correlation matrix to understand how are the features interrelated with each other.

Also to get an idea which are the most important features in this data.

In [ ]:
sns.heatmap(newdf.corr(), annot=True, cmap = 'Reds')
plt.show()

Observation:

There's correlation among host_id to reveiws_per_month & availability_365.

Also there's noticiable correlation between min_nights to no_of_listings_count & availability_365. Price also shows some correlation with availability_365 & host_listings_count.

no_of_reviews and reviews_per_month gives almost the same information. so we can carry out analysis with any of the two variable. Also, no_of_reviews is correlated to availability_365!

In [ ]:
newdf.info()

#**Univariate Analysis**
##Categorical Column Analysis
##What listing names are the most no of listings in Airbnb?

In [ ]:
df.name.nunique()

In [ ]:
df.name.value_counts

In [ ]:
top_5_listings_count_df=df['name'].value_counts()[:5].reset_index()
top_5_listings_count_df.rename(columns={'index':'Listings_on_Airbnb','name':'Total_listings'},inplace=True)
top_5_listings_count_df

In [ ]:
ax=sns.barplot(x='Listings_on_Airbnb',y='Total_listings',data=top_5_listings_count_df)
for item in ax.get_xticklabels():
    item.set_rotation(90)

Answer: Hillside Hotel is found to have listed more listings in entire Airbnb, followed by Home away from Home.

## What are the host ids with most listings counts?

In [ ]:
df.host_id.nunique()

In [ ]:
df.host_id.value_counts

In [ ]:
top_10_host_id=df['host_id'].value_counts()[:10].reset_index()
top_10_host_id.rename(columns={'index':'Host_Id','host_id':'Total_listings'},inplace=True)
top_10_host_id

In [ ]:
ax=sns.barplot(x='Host_Id',y='Total_listings',data=top_10_host_id)
for item in ax.get_xticklabels():
  item.set_rotation(90)

Answer: Host id 219517861 is found to have listed more listings in entire NYC with 327, followed by 107434423.

#Find Out the top 10 hosts on the basis of no of listings in entire Airbnb?

In [ ]:
df.host_name.nunique()

In [ ]:
df.host_name.value_counts()

In [ ]:
top_hosts=df['host_name'].value_counts()[:10].reset_index()
top_hosts.rename(columns={'index':'host_name','host_name':'Total_listings'},inplace=True)
top_hosts

In [ ]:
ax=sns.barplot(x='host_name',y='Total_listings',data=top_hosts)
for item in ax.get_xticklabels():
  item.set_rotation(90)

Answer: Michael is found to have listed more listings in entire NYC with 417, followed by David.

# What are the Neighbourhood Groups with most listings counts?

In [ ]:
df.neighbourhood_group.nunique()

In [ ]:
df.neighbourhood_group.value_counts()

In [ ]:
top_neighbourhood_group = df.neighbourhood_group.value_counts().reset_index()
top_neighbourhood_group.rename(columns={'index':'Neighbourhodd_Group','neighbourhood_group':'Total_Listings'},inplace=True)
top_neighbourhood_group

In [ ]:
ax=sns.barplot(x='Neighbourhodd_Group',y='Total_Listings',data=top_neighbourhood_group)

Answer: Manhattan is found to have high listed neighbourhood group more listings in entire NYC, followed by Brooklyn.

# Find Out the top 10 neighbourhood on the basis of no of listings in entire Airbnb?

In [ ]:
df.neighbourhood.nunique()

In [ ]:
df.neighbourhood.value_counts()

In [ ]:
top_neighbourhood=df['neighbourhood'].value_counts()[:10].reset_index()
top_neighbourhood.rename(columns={'index':'neighbourhood','neighbourhood':'Total_listings'},inplace=True)
top_neighbourhood

In [ ]:
ax=sns.barplot(x='neighbourhood',y='Total_listings',data=top_neighbourhood)
for item in ax.get_xticklabels():
  item.set_rotation(90)

Answer: Williamsburg is found to have listed more listed neighbourhood in entire NYC with 3920, followed by Bedford-Stuyvesant.

# What are the room types with most listings counts?

In [ ]:
df.room_type.nunique()

In [ ]:
df.room_type.value_counts()

In [ ]:
top_rooms = df.room_type.value_counts().reset_index()
top_rooms.rename(columns={'index':'room_type','room_type':'Total_Listings'},inplace=True)
top_rooms

In [ ]:
ax=sns.barplot(x='room_type',y='Total_Listings',data=top_rooms)

Answer: Entire home/Apt is found to have high listed room type with more listings in entire NYC, followed by Private room.

#Numerical Column Analysis

In [ ]:
newdf.describe()

**Observation :**

**Price:**



1.   1.The least price on listings is 10 dollars.
2.The Average price on listings is 152 dollars
3.The Max price on listings is 10,000 dollars.






**minimum_nights:**


1.   The min nights spent were 1.
2.The Average nights spent were 7.
3.The max nights spent were 1250.





**number_of_reviews:**



1.   The min number of reviews were 0.
2.The number of reviews spent were 23.
3.The max number of reviews were 629.



**reviews_per_month:**



1.   The min number of reviews per month were 0.
2.The Average number of reviews per month were around 1.
3.The max number of reviews per month were 58.



**calculated_host_listings_count:**



1. The min number of listings by a host were 1.
2.The Average number of listings by a host 7.
3.The max number of listings by a host were 327.


**availability_365:**



1.   The min number of listings availability through year were 0.
2.The Average number of listings availability through year around 113.
3.The max number of listings availability through year were 365.


#**top 25 used words for listing names**

In [ ]:
#initializing empty list where we are going to put our name strings
_names_=[]
#getting name strings from the column and appending it to the list
for name in df.name:
    _names_.append(name)

#setting a function that will split those name strings into separate words   
def split_name(name):
    spl=str(name).split()
    return spl

#initializing empty list where we are going to have words counted
_names_for_count_=[]

#getting name string from our list and using split function, later appending to list above
for x in _names_:
    for word in split_name(x):
        word=word.lower()
        _names_for_count_.append(word)

In [ ]:
#we are going to use counter
from collections import Counter
#let's see top 25 used words by host to name their listing
# Counting most common words
count_words = Counter(_names_for_count_).most_common()
count_words[:25]

In [ ]:
# Cleaning the List by removing prepositions
items_to_remove = {('in', 16733), ('the', 3869), ('to', 3827), ('of', 2993), ('-', 2272), ('a', 1909)}
top_25_cleaned = [e for e in count_words if e not in items_to_remove]
top_25 = top_25_cleaned[:25]
top_25

In [ ]:
#now let's put our findings in dataframe for further visualizations
sub_w=pd.DataFrame(top_25)
sub_w.rename(columns={0:'Words', 1:'Count'}, inplace=True)

In [ ]:
#we are going to use barplot for this visualization
viz_words=sns.barplot(x='Words', y='Count', data=sub_w)
viz_words.set_title('Counts of the top 25 used words for listing names')
viz_words.set_ylabel('Count of words')
viz_words.set_xlabel('Words')
viz_words.set_xticklabels(viz_words.get_xticklabels(), rotation=80)

##**word cloud visualization method**

In [ ]:
#Importing WordCloud library
from wordcloud import WordCloud, ImageColorGenerator
text = ' '.join(str(n) for n in sub_w.Words)

#Generating wordcloud image
wordcloud = WordCloud(max_font_size=80, colormap='hsv', width=400, height=150).generate(text)
plt.figure(figsize=(12,8))

#Displaying the image
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

##**Bi-variate Analysis**
###**No of listings in Neighbourhood Group by Room types**

In [ ]:
ax = sns.countplot(data=newdf,y="room_type", hue="neighbourhood_group",palette='bright')

total = len(df['room_type'])
for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/total)
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(percentage, (x, y))

plt.title('count of each room types in entire NYC')
plt.xlabel('Rooms')
plt.xticks(rotation=90)
plt.ylabel('Room Counts')

plt.show()

Manhattan has more listed properties with Entire home/apt around 27% of total listed properties followed by Brooklyn with around 19.6%.

Private rooms are more in Brooklyn as in 20.7% of the total listed properties followed by Manhattan with 16.3% of them. While 6.9% of private rooms are from Queens.

Very few of the total listed have shared rooms listed on Airbnb where there's negligible or almost very rare shared rooms in Staten Island and Bronx.

We can infer that Brooklyn,Queens,Bronx has more private room types while Manhattan which has the highest no of listings in entire NYC has more Entire home/apt room types.

# **No of hosts in location group by Neighbourhood Group**

In [ ]:
#Apply group by operation on neighbourhood_group for find the number of host according to location
no_of_host_per_location = newdf.groupby('neighbourhood_group',as_index=False)['host_id'].count().sort_values(['host_id'], ascending=False).rename(columns={'neighbourhood_group':'Location','host_id':'Host'})
no_of_host_per_location

In [ ]:
no_of_host_per_location =newdf.groupby('neighbourhood_group')['host_id'].count()
no_of_host_per_location.plot()
plt.title('Number of host per location')
plt.ylabel('Host')
plt.xlabel('Location')
plt.legend(labels = ['No. of Host'])

Manhattan is most focused place for host to do their business

## Now let's understand the distribution of price along neighbourhood groups
Distribution of price across Manhattan,Brooklyn,Queens,Bronx & Staten Island

In [ ]:
ax= sns.violinplot(x='neighbourhood_group',y='price',data=iqrdf)

**Manhattan being the most costliest place to live in, have price more than 140 USD followed by Brooklyn with around 80 USD on an average for the listings.**

**Queens, Staten Island are on the same page with price on listings** 

In [ ]:
#looking for the average price each neighbourhood groups holds in a seaborn pointplot
sns.pointplot(x = 'neighbourhood_group', y='price', data=iqrdf, estimator=mean)

We can see that Manhattan has the highest(the costliest) average of price ranging to ~150dollars/day followed by Brooklyn

Queens and Staten Island have same average price/day, while Bronx comes out to be the cheapest neighbourhood group in NYC.

Well, let's deep dive into cost of living as it is the most important factor for someone willing to rent/stay using Airbnb.

# **The highest and lowest rent according to neighbourhood group**

In [ ]:
#Get the highest rent according to neighbourhood group
max_price_df = newdf.groupby('neighbourhood_group',as_index=False)['price'].max().sort_values(['price'],ascending = False).rename(columns = {'price':'Maximum price','neighbourhood_group':'Location'})
max_price_df

**Brooklyn,Manhattan& Queens are highest renting neighbourhood groups**

In [ ]:
#Get the lowest rent according to neighbourhood group
min_price_df = newdf.groupby('neighbourhood_group',as_index=False)['price'].min().sort_values(['price'],ascending = True).rename(columns = {'price':'Minimum price','neighbourhood_group':'Location'})
min_price_df

**Brooklyn,Manhattan,Brooklyn & queens are the lowest renting neighbourhood groups**

In [ ]:
#Get the combine dataframe of minimum and maximum price according to location
merge_price_df = pd.merge(max_price_df, min_price_df, on='Location')
merge_price_df

# **The highest and lowest rent according to neighbourhood**

In [ ]:
#Get the highest rent according to neighbourhood
max_price_neigh_df = newdf.groupby('neighbourhood',as_index=False)['price'].max().sort_values(['price'],ascending = False).rename(columns = {'price':'Minimum price'})
max_price_neigh_df[:10]

**Upper west side,Greenpoint & Astoria are highest renting neighbourhoods**

In [ ]:
#Get the lowest rent according to neighbourhood
min_price_neigh_df = newdf.groupby('neighbourhood',as_index=False)['price'].min().sort_values(['price'],ascending = True).rename(columns = {'price':'Minimum_price'})
min_price_neigh_df[min_price_neigh_df['Minimum_price']==10]

In [ ]:
len(min_price_neigh_df[min_price_neigh_df['Minimum_price']==10])

**There are 15 neighbourhoods where the lowest cost is 10 dollars**

In [ ]:
#Get the combine dataframe of minimum and maximum price according to location
merge_price_neigh_df = pd.merge(max_price_neigh_df, min_price_neigh_df, on='neighbourhood')
min_price_neigh_df[min_price_neigh_df['Minimum_price']==10]

# **The highest and lowest rent according to property**

In [ ]:
#Get the highest rent according to property
max_price_name_df = newdf.groupby('name',as_index=False)['price'].max().sort_values(['price'],ascending = False)
max_price_name_df

**Luxury 1 bedroom apt. -stunning Manhattan views ,Furnished room in Astoria apartment & 1-BR Lincoln Center are highest renting property with 10,000 dollars**

In [ ]:
#Get the lowest rent according to location
min_price_name_df = newdf.groupby('name',as_index=False)['price'].min().sort_values(['price'],ascending = True)
min_price_name_df

In [ ]:
min_price_name_df[min_price_name_df['price']==10]

In [ ]:
len(min_price_name_df[min_price_name_df['price']==10])

**These 17 listings are the cheapest listings according to price on nyc**

# **Average for how many nights people stayed in each room_types**

In [ ]:
newdf.groupby('room_type')['minimum_nights'].mean().plot(kind='barh',color='r')
plt.title('Min Stays in different room types listed on Airbnb ')
plt.xlabel('Room types')
plt.ylabel('Min Stays')

**Looking at these it clearly indicates that people mostly prefer living in an entire home/apt on an average of more than 8 nights followed by guests who stayed inshared room where average stay is 6-7 nights.**

# **Multi-variate Analysis**
## **Now let's understand the distribution of price(without extreme values) along neighbourhood groups with room types** 

In [ ]:
iqrdf['neighbourhood_group'].value_counts()

In [ ]:
#Unstack the group by information for plot the graph
avg_preffered_price_df = iqrdf.groupby(['neighbourhood_group','room_type'])['price'].mean().unstack()
avg_preffered_price_df

In [ ]:
#Graph of average price of property according to location
avg_preffered_price_df.plot.bar()

**Manhattan is leading in all three roomtypes on average price across nyc**

# **Now, Let's check the expensive & cheapest listings with respect to prices leading in entire Airbnb!**

**The costliest?**

In [ ]:
newdf.nlargest(5,'price')[['name','neighbourhood_group','neighbourhood','host_name','room_type']]

**Queens is top the list followed by brooklyn. Looks like the all the top 3,4 & 5 costliest listings belongs to Manhattan itself(being the most costliest place of living in Airbnb)**

**The cheapest?**

In [ ]:
newdf.sort_values(by='price',ascending=True)[['name','neighbourhood_group','neighbourhood','host_name','room_type']][:5]

**Also, the cheapest listings mostly comes from Manhatten and Queens.**

# **Top neighbourhoods in Airbnb with respect to average price/day of Airbnb listings:**

**let's check for the neighbourhoods with listings having highest average price/day in each neighbourhood groups of Airbnb!**

In [ ]:
df_ngrp= iqrdf.groupby(['neighbourhood_group','neighbourhood'])['price'].mean().reset_index()

In [ ]:
top_neigh_each= df_ngrp.loc[df_ngrp.groupby('neighbourhood_group')['price'].idxmax()].sort_values(by='price',ascending=False)
top_neigh_each

Among the top neighbourhoods in each neighbourhood groups, top 2 of them namely: **Fort Wadsworth & Tribeca, origins from Staten Island & Manhattan.**

**Seagate** from **Brooklyn** and **Riverdale** from **Bronx**. **Neponsit** from **Queens**.

In [ ]:
top_neigh_each.plot.bar(x='neighbourhood', rot=90, title='Top Neighbourhoods in each neighbourhood groups in NYC')
plt.rcParams["figure.figsize"] = (20,3)
plt.xlabel('neighbourhoods')
plt.ylabel('price')

## **Let's checkout the costliest listings in each neighbourhood groups & their respective hosts:**

In [ ]:
costliest_df= newdf.loc[newdf.groupby(['neighbourhood_group'])['price'].idxmax()][['name','neighbourhood_group','host_name','price']]

In [ ]:
costliest_df.sort_values(by='price',ascending=False)

Looks like the top 2 costliest listings: Luxury 1 bedroom apt. -stunning Manhattan views & 1-BR Lincoln Center were from Brooklyn and Manhattan, hosted by Erin and jelena respectively.

**Also, the cheapest listings in each neighbourhood groups & their respective hosts:**

In [ ]:
cheapest_df= newdf.loc[newdf.groupby('neighbourhood_group')['price'].idxmin()][['neighbourhood_group','name','host_name','price']]

In [ ]:
cheapest_df.sort_values(by='price',ascending=True)

**Looks almost all the neighbourhood groups with the lowest price falls in the same price range for the cheapest listings.**

## **Room_types vs price on different neighbourhood groups!**

In [ ]:
ax = sns.barplot(x="neighbourhood_group", y="price", hue="room_type", data=newdf,ci=None)

Looks like a property/listing with **Entire home/apt** as room_type wins the show at NYC followed by private rooms.

**Manhattan** has the highest price for room types with **Entire home/apt** ranging to nearly **240 USD/night**, followed by Private room with **110 USD/night**. And it's obvious being the most expensive place to live in!

## **top 10 most reviewed listings in NYC!**

In [ ]:
top10_reviewed_listings= newdf.nlargest(10,'reviews_per_month')
top10_reviewed_listings[['name','reviews_per_month','neighbourhood_group']]

**So, this are the listings with highest number of reviews, the highest being: Enjoy great views of the City in our Deluxe Room! with 58 reviews/month followed by Great Room in the heart of Times Square! with a total of 27 reviews/month. Top 2 are from Manhattan so we can decide the best suited place for a comfortable stay but the prices are high to afford.**

Also, Lets look how monthly reviews varies with room types in each neighbourhood groups!

**Average Price In Top 10 Reviewed Listings Per Night**

In [ ]:
price_avrg=top10_reviewed_listings.price.mean()
print('Average price per night: {}'.format(price_avrg))

## **Let's find the top 10 reviewed hosts on the basis of reviews/month**

In [ ]:
#there are cases where same hosts are hosting multiple properties in same/different areas.
reviews_df=top10_reviewed_listings.groupby('host_name')['reviews_per_month'].mean() 
reviews_df=reviews_df.reset_index().sort_values(by='reviews_per_month',ascending=False)
reviews_df.plot(x='host_name',y='reviews_per_month',kind='bar',color='r')
plt.ylabel('Reviews counts')
plt.xlabel('Host names')
plt.title('Top 10 Reviews/month received by hosts')
plt.rcParams["figure.figsize"] = (15,10)

Row NYC holds the title as the most reviewed host with more than 40 reviews/month on average.

## **Hosts with most no of listings in NYC(on the basis of host listings count):**

In [ ]:
host_with_most_listings= newdf.groupby(['host_name','neighbourhood_group'])['calculated_host_listings_count'].sum().reset_index()
largest_listings_df= host_with_most_listings.nlargest(10,'calculated_host_listings_count')
largest_listings_df

**So, Sonder(NYC) leads the most expensive neighbourhood group in NYC with around 106929 properties listed followed by Blueground and Kara**.

**Also, all 3 of them has their listings on Manhattan.**

## **Hosts with most listings in each neighbourhood group.**

In [ ]:
hosts_df=host_with_most_listings.groupby(['neighbourhood_group'])['calculated_host_listings_count'].transform(max) == host_with_most_listings['calculated_host_listings_count']
host_with_most_listings[hosts_df].sort_values(by='calculated_host_listings_count',ascending=False)

**So, Sonder (NYC), Kazuya, Vida, Vie and Amarjit** are the hosts with most no of listings in each neighbourhood groups in entire NYC!

## **Room_types and their relation with availability in different neighbourhood groups!**

In [ ]:
f,ax = plt.subplots(figsize=(15,8))
ax=sns.boxplot(x='neighbourhood_group',y='availability_365',data=newdf,palette="bright")
plt.title("Neighbourhood Group vs. Room Availabilty")
plt.xlabel('Neighbourhood groups')
plt.ylabel('Availability(365)')
plt.show()

**Looking at the above categorical box plot we can infer that the listings in Staten Island seems to be more available throughout the year to more than 300 days. On an average, these listings are available to around 210 days every year followed by Bronx where every listings are available for 150 on an average every year.**

## **Neighbourhood groups visualization with latitude and longitude!**

In [ ]:
#trying to find where the coordinates belong from the latitude and longitude
sns.set(rc={"figure.figsize": (10, 8)})
ax= sns.scatterplot(data=newdf, x="longitude", y="latitude",hue='neighbourhood_group',palette='bright')
ax.set_title('Location Co-ordinates')

## **Types of Rooms visualization with latitude and longitude!**

In [ ]:
sns.set(rc={"figure.figsize": (10, 8)})
ax= sns.scatterplot(x="longitude", y="latitude",hue="room_type",data=newdf,palette='muted')
ax.set_title('Distribution of type of rooms across NYC')

## **price variations across these co-ordinates in a more clear way**

In [ ]:
lat_long=iqrdf.plot(kind='scatter', x='longitude', y='latitude', label='availability_365', c='price',
                  cmap=plt.get_cmap('jet'), colorbar=True, alpha=0.4, figsize=(10,8))
lat_long.legend()

**We can infer that there are high range of prices across Manhattan being the most costliest place to stay in NYC**

**It is important to note that we had to drop some extremely high values as they are treated as outliers for our analysis.**

# **Conclusion:**





1.   Hillside Hotel is found to have listed more listings in entire NYC, followed by Home away from Home. 
2. Host id 219517861 is found to have listed more listings in entire NYC with 327, followed by 107434423

3. Michael is found to have listed more listings in entire NYC with 417, followed by David. 
4. Manhattan is found to have high listed neighbourhood group more listings in entire NYC, followed by Brooklyn.

5. Williamsburg is found to have listed more listed neighbourhood in entire NYC with 3920, followed by Bedford-Stuyvesant.

6. Entire home/Apt is found to have high listed room type with more listings in entire NYC, followed by Private room.

7. Manhattan has more listed properties with Entire home/apt around 27% of total listed properties followed by Brooklyn with around 19.6% 
8. Private rooms are more in Brooklyn as in 20.7% of the total listed properties followed by Manhattan with 16.3% of them. While 6.9% of private rooms are from Queens.

9. Manhattan is most focused place for host to do their business

10. Manhattan has the highest(the costliest) average of price ranging to -150dollars/day followed by Brooklyn

11. Queens and Staten Island have same average price/day, while Bronx comes out to be the cheapest neighbourhood group in NYC. 
12. Brooklyn Manhattan & Queens are highest renting neighbourhood groups

13. Brooklyn Manhattan Brooklyn & queens are the lowest renting neighbourhood groups

14. Upper west side,Greenpoint & Astoria are highest renting neighbourhoods

15. There are 15 neighbourhoods where the lowest cost is 10 dollars

16. Luxury 1 bedroom apt. -stunning Manhattan views Furnished room in Astoria apartment & 1-BR Lincoln Center are highest renting property with 10,000 dollars 
17. people mostly prefer living in an entire home/apt on an average of more than 8 nights followed by guests who stayed inshared room where average stay is 6-7 nights.

18. Manhattan is leading in all three roomtypes on average price across nyc
19. Staten Island seems to be more available throughout the year to more than 300 days. On an average, these listings are available to around 210 days every year followed by Bronx where every listings are available for 150 on an average every year.




